# start

In [1]:
import time
import asyncio
import threading
import inspect
import sys

# trong jupyter phai co moi run duoc asyncio
import nest_asyncio
nest_asyncio.apply()

#-----------------------------------------------------------------------------#


def child_zero():# funciton='child'
    return f"{inspect.stack()[0].function}"

def child_one():# function='name tu func goi no'
    return f"{inspect.stack()[1].function}"

def whoami_zero(): 
    return sys._getframe(0).f_code.co_name

def whoami_one(): 
    return sys._getframe(1).f_code.co_name

def add_lines(_func, /, *, line=1):
    
    def decorator(func):
        
        def wrapper(*args, **kwds):
            print(end='\n'*line)
            return func(*args, **kwds)
        
        return wrapper
    
    if _func:
        return decorator(_func)
    else:
        return decorator

# asyncio

## asyncio.gather(*coro)

In [2]:
import time
import asyncio
import threading

# trong jupyter phai co moi run duoc asyncio
import nest_asyncio
nest_asyncio.apply()

# coro = my_coro() # da run function
"""
    dùng asyncio.gather(*coro): auto connect các coro với PIPE LON, 
    run theo vị trí chúng ta pass các coro vào trong gather(*coro)
"""

async def animal():
    print(f"{'animal':$^50}")
    await asyncio.sleep(2) #jumpto human
    print(f"{'end sleep(2)':->30}")
    
    await dog() # run in dog() gap await jumpto tiep
    print(f"{'animal end':$^50}")
    return 'animal'

async def dog():
    for i in range(1, 3):
        print(f'dog {i} start')
        await asyncio.sleep(1)
        print(f'dog {i} end')


async def human():
    print(f"{'human':#^50}")
    await thong() # child coro 
    print(f"{'human end':#^50}")
    return 'human'

async def thong():
    for i in range(1, 5):
        print(f'thong {i} start')
        await child_thong(i)
        print(f'thong {i} end')

async def child_thong(i):
    print(f'child_thong {i} start')
    await asyncio.sleep(1)
    print(f'child_thong {i} end')
        

async def machine():
    print(f"{'machine':*^50}")
    await computer()
    print(f"{'machine end':*^50}")
    return 'machine'

async def computer():
    for i in range(1, 5):
        print(f'computer {i} start')
        await asyncio.sleep(1)
        print(f'computer {i} end')


async def main(*coro):
    """
        await asyncio.gather(animal(), human(), machine()): run theo thứ tự arguments trong gather()
        kết nối 3 coro = pipe child, vao PIPE LỚN, vẫn có thứ tự run là 
        gặp await: ANIMAL jumpto HUMAN jumto MACHINE jumto ANIMAL jumpto HUMAN jumto MACHINE ... chừng nào hết thì thôi
        NOTE: if ANIMAL RUN; trong nó có await child_coro1() thì run nó, trong nó có await cihld_coro2() thi run tiếp 
        nếu gặp await asyncio.sleep(number) thi jumto pipe nhỏ
        program hiên tại có 3 pipe nhỏ theo thứ tự là: animal - human - machine
        
        
        keyword await coro() có chức năng giống với 
        write<-, <-read của golang(nhưng golang gặp số LẺ mới jumpto, số CHẴN thì run qua lun
    """
    values = await asyncio.gather(animal(), human(), machine())
    return values

async def test(*coro):
    values = await asyncio.gather(*coro)
    return values

def run(main, *coro):
    s = time.perf_counter()

    asyncio.run(main(*coro))
    # asyncio.run(task1())
    # asyncio.run(main_two())

    elapsed = time.perf_counter() - s
    print(f"{elapsed:0.2f} seconds.")

run(test, human(), animal())

######################human#######################
thong 1 start
child_thong 1 start
$$$$$$$$$$$$$$$$$$$$$$animal$$$$$$$$$$$$$$$$$$$$$$
child_thong 1 end
thong 1 end
thong 2 start
child_thong 2 start
------------------end sleep(2)
dog 1 start
child_thong 2 end
thong 2 end
thong 3 start
child_thong 3 start
dog 1 end
dog 2 start
child_thong 3 end
thong 3 end
thong 4 start
child_thong 4 start
dog 2 end
$$$$$$$$$$$$$$$$$$$$animal end$$$$$$$$$$$$$$$$$$$$
child_thong 4 end
thong 4 end
####################human end#####################
4.02 seconds.


## asyncio.create_task(coro())

In [3]:
import time
import asyncio
import threading

# trong jupyter phai co moi run duoc asyncio
import nest_asyncio
nest_asyncio.apply()

"""
    dùng asyncio.create_task(coro()) : connect thủ công  PIPE LON, 
    và phải nằm trên đoạn code jumpto: asyncio.sleep(number)
    
    dùng asyncio.gather(*coro): auto connect các coro với PIPE LON, 
    run theo vị trí chúng ta pass các coro vào trong gather(*coro)
"""

async def task1():
    print('Send first Email')
    
    #asyncio.create_task(task2())
    #asyncio.create_task(task3())

    await asyncio.sleep(3)
   
    print('First Email reply')

async def task2():
    print('Send second Email')
    #asyncio.create_task(task3())
    await asyncio.sleep(1)
    print('Second Email reply')

async def task3():
    print('Send third Email')
    await asyncio.sleep(2)
    print('Third Email reply')
    
async def main():
    # run la dau task1() await jumpto, task2() await jumpto, task3() await, sau do quay lai run func nao co await
    # thoi gian cho =run it nhat truoc
    await asyncio.gather(task1(), task2(), task3())

asyncio.run(main())

Send first Email
Send second Email
Send third Email
Second Email reply
Third Email reply
First Email reply


In [4]:
import time
import asyncio
import threading

# trong jupyter phai co moi run duoc asyncio
import nest_asyncio
nest_asyncio.apply()


async def task1():
    print('Send first Email')
    asyncio.create_task(task2())
    asyncio.create_task(task3())
    # asyncio.create_task(func()) phai nam tren await asyncio.sleep(3)
    
    # task1(), task2() se run xong xong voi task1()
    # lan run dau tien,se run theo thu tu code asyncio.create_task(func())
    # lan thu 2, se run func nao co thoi gian await it nhat run truoc, tuc END
    
    
    await asyncio.sleep(3) # run sleep(2), jumpto task2()
        
    # end task1 
    print('First Email reply')

async def task2():
    print('Send second Email')
    await asyncio.sleep(1) # run sleep(1), jumpto task3()
    # end task2
    print('Second Email reply')

async def task3():
    print('Send third Email')
    await asyncio.sleep(2)
    # end task2
    print('Third Email reply') #end run tiếp về end task1 ,2 ,3
    
async def main():
    await asyncio.gather(task1(), task2(), task3())

asyncio.run(task1())

Send first Email
Send second Email
Send third Email
Second Email reply
Third Email reply
First Email reply


# coroutine basic, async_generator

In [5]:
import asyncio
import nest_asyncio
nest_asyncio.apply()

async def coroutine():
    return 'coroutine'

'''
    call use: 
    async for item in genfunc()
'''
async def genfunc():
    yield 1
    yield 2
    yield 3

async def main():
    '''coroutine basic return dung keywork await'''
    coro = await coroutine()

    async for v in genfunc():
        print(v)

    print(coro)


asyncio.run(main())

1
2
3
coroutine


# AsyncIteratorWrapper

In [6]:
import asyncio
import nest_asyncio
nest_asyncio.apply()

class AsyncIteratorWrapper:
  def __init__(self, obj):
    self._it = iter(obj)

  def __aiter__(self):
    return self

  async def __anext__(self):
    try:
      value = next(self._it)
    except StopIteration:
      raise StopAsyncIteration
    return value

'''dung async for item in  AsyncIteratorWrapper('abcd')'''
async def main():
 async for letter in AsyncIteratorWrapper("abcd"):
  print(letter)

asyncio.run(main())

a
b
c
d


# Context Manager

In [7]:
class Indenter:
    def __init__(self):
        self.level = -1

    def __enter__(self):
        self.level += 1
        return self

    def __exit__(self, exc_type, exc_value, exc_tb):
        self.level -= 1

    def print(self, text):
        print("    " * self.level + text)
'''
 with Indenter() as indent:
 indent = Indenter().__enter__()=self

'''

'\n with Indenter() as indent:\n indent = Indenter().__enter__()=self\n\n'

In [8]:
with Indenter() as indent:
    indent.print("hi!")
    with indent:
        indent.print("hello")
        with indent:
            indent.print("bonjour")
    indent.print("hey")

hi!
    hello
        bonjour
hey


In [9]:
from contextlib import contextmanager

@contextmanager
def hello_context_manager():
    print("Entering the context...")
    yield "Hello, World!"
    print("Leaving the context...")


with hello_context_manager() as hello:
    print(hello)

Entering the context...
Hello, World!
Leaving the context...


# Asynchronous Context Manager

In [10]:
import aiohttp
import asyncio
import nest_asyncio
nest_asyncio.apply()

class AsyncSession:
    def __init__(self, url):
        self._url = url

    async def __aenter__(self):
        self.session = aiohttp.ClientSession()
        response = await self.session.get(self._url)
        return response

    async def __aexit__(self, exc_type, exc_value, exc_tb):
        await self.session.close()

async def check(url):
    async with AsyncSession(url) as response:
        print(f"{url}: status -> {response.status}")
        html = await response.text()
        print(f"{url}: type -> {html[:17].strip()}")

async def main():
    await asyncio.gather(
        check("https://realpython.com"),
        check("https://pycoders.com"),
    )

asyncio.run(main())

https://pycoders.com: status -> 200
https://pycoders.com: type -> <!doctype html>
https://realpython.com: status -> 200
https://realpython.com: type -> <!doctype html


In [11]:
async def say_after(delay, what):
    print(f"START {what} {delay:.>10}")
    await asyncio.sleep(delay)
    print(f"END {what:.>10}", end='\n'*2)

# python 3.7
async def say_after_main():
    """ code run tu tren xuong, khi gap await thi quay len treb chay theo cac create_task() tu tren xuong duoi"""


    print(f"started at {time.strftime('%X')}")

    #two create tasks -> dung asyncio.gather co the bo buoc #two
    # task1 = asyncio.create_task(say_after(5,'5'))
    # task2 = asyncio.create_task(say_after(3,'3'))
    # task4 = asyncio.create_task(say_after(7,'7'))
    # task3 = asyncio.create_task(say_after(1,'1'))
    """ await chờ task2 ket thuc rồi chạy het chuong trinh, cac task co thoi gian lau hon se bo qua
     """
    # await cho_5p => t chờ tui bay 5p, het 5p t chay het func cua tao
    # await tasks = run asyncio.create_task(fun())
    # await task1
    # await task2
    # await task4

    #three
    await asyncio.gather(say_after(9,'9'), say_after(3,'3'), say_after(7,'7'),)

    print(f"finished at {time.strftime('%X')}")
# python 3.8
# asyncio.run(say_after_main()) #three

#=============================================================================#

# old

In [12]:
# python 3.6 va cu hon
def say_after_until():
    """ goi ten ham say_after_until() khong can asyncio.run(say_after_until())"""


    print(f"started at {time.strftime('%X')}")

    loop = asyncio.get_event_loop()

    task1 = loop.create_task(say_after(5,'5'))
    task2 = loop.create_task(say_after(3,'3'))
    task3 = loop.create_task(say_after(7,'7'))
    task4 = loop.create_task(say_after(1,'1'))
    loop.run_until_complete(asyncio.gather(task2))
    # loop.run_until_complete(asyncio.gather(task1, task4, task3, task2))
    # run theo luc loop.create_task, chu ko phai theo asyncio.gather(*task)

    print(f"finished at {time.strftime('%X')}")
# say_after_until()
#=============================================================================#


def thread_test(name='one',label='#', stop=30, delay=1):
    print(f"{name} let go!!!")

    for i in range(stop):
        print(f'{name}: {i:{label}>20}')
        time.sleep(delay)

def thread_main():

    r1 = threading.Thread(target=thread_test, args=('one','#',16,4))
    r2 = threading.Thread(target=thread_test, args=('two','-',40,1))

    r2.start()
    r1.start()

    threads = [r1, r2]
    for r in threads:
        r.join()

    print('end threading')
    

if __name__ == '__main__':
    pass